<a href="https://colab.research.google.com/github/Miguel231/Computing-Acceleration/blob/main/3-3-4-TFLiteConverter.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
# Install TensorFlow 2.10
!pip install numpy==1.26
!pip install tensorflow

In [6]:
import tensorflow as tf
import numpy as np
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense

In [7]:
l0 = Dense(units=1, input_shape=[1])
model = Sequential([l0])
model.compile(optimizer='sgd', loss='mean_squared_error')

xs = np.array([-1.0, 0.0, 1.0, 2.0, 3.0, 4.0], dtype=float)
ys = np.array([-3.0, -1.0, 1.0, 3.0, 5.0, 7.0], dtype=float)

model.fit(xs, ys, epochs=500)

print(model.predict(np.array([10.0])))
print("Here is what I learned: {}".format(l0.get_weights()))

Epoch 1/500
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 302ms/step - loss: 17.6396
Epoch 2/500
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 70ms/step - loss: 14.1323
Epoch 3/500
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 86ms/step - loss: 11.3677
Epoch 4/500
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 165ms/step - loss: 9.1875
Epoch 5/500
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 128ms/step - loss: 7.4672
Epoch 6/500
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 133ms/step - loss: 6.1089
Epoch 7/500
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 134ms/step - loss: 5.0354
Epoch 8/500
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 105ms/step - loss: 4.1861
Epoch 9/500
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 94ms/step - loss: 3.5134
Epoch 10/500
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 182ms/step - loss: 2.9795
Epoch 11/500
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 157ms/step - loss: 2.5551
Epoch 12/500
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 199ms/step - loss: 2.2169
Epoch 13/500
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 247ms/step - loss: 1.9465
Epoch 14/500
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 129ms/step - loss: 1.7297
Epoch 15/500
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 147ms/step - loss: 1.5550
Epoc

In [12]:
export_dir = 'saved_model/1'
model.export(export_dir)  # <-- This replaces tf.saved_model.save or model.save for SavedModel


Saved artifact at 'saved_model/1'. The following endpoints are available:

* Endpoint 'serve'
  args_0 (POSITIONAL_ONLY): TensorSpec(shape=(None, 1), dtype=tf.float32, name='keras_tensor_2')
Output Type:
  TensorSpec(shape=(None, 1), dtype=tf.float32, name=None)
Captures:
  135259044245200: TensorSpec(shape=(), dtype=tf.resource, name=None)
  135259044246544: TensorSpec(shape=(), dtype=tf.resource, name=None)


In [13]:
# Convert the model.
converter = tf.lite.TFLiteConverter.from_saved_model(export_dir)
tflite_model = converter.convert()

In [14]:
import pathlib
tflite_model_file = pathlib.Path('model.tflite')
tflite_model_file.write_bytes(tflite_model)

1104

In [15]:
# Load TFLite model and allocate tensors.
interpreter = tf.lite.Interpreter(model_content=tflite_model)
interpreter.allocate_tensors()

# Get input and output tensors.
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()
print(input_details)
print(output_details)

[{'name': 'serving_default_keras_tensor_2:0', 'index': 0, 'shape': array([1, 1], dtype=int32), 'shape_signature': array([-1,  1], dtype=int32), 'dtype': <class 'numpy.float32'>, 'quantization': (0.0, 0), 'quantization_parameters': {'scales': array([], dtype=float32), 'zero_points': array([], dtype=int32), 'quantized_dimension': 0}, 'sparsity_parameters': {}}]
[{'name': 'StatefulPartitionedCall_1:0', 'index': 3, 'shape': array([1, 1], dtype=int32), 'shape_signature': array([-1,  1], dtype=int32), 'dtype': <class 'numpy.float32'>, 'quantization': (0.0, 0), 'quantization_parameters': {'scales': array([], dtype=float32), 'zero_points': array([], dtype=int32), 'quantized_dimension': 0}, 'sparsity_parameters': {}}]


/usr/local/lib/python3.12/dist-packages/tensorflow/lite/python/interpreter.py:457: UserWarning:     Warning: tf.lite.Interpreter is deprecated and is scheduled for deletion in
    TF 2.20. Please use the LiteRT interpreter from the ai_edge_litert package.
    See the [migration guide](https://ai.google.dev/edge/litert/migration)
    for details.
    
  warnings.warn(_INTERPRETER_DELETION_WARNING)


In [16]:
to_predict = np.array([[10.0]], dtype=np.float32)
print(to_predict)
interpreter.set_tensor(input_details[0]['index'], to_predict)
interpreter.invoke()
tflite_results = interpreter.get_tensor(output_details[0]['index'])
print(tflite_results)

[[10.]]
[[18.981104]]
